In [ ]:
# !pip install ipywidgets
# !jupyter nbextension enable --py widgetsnbextension

In [ ]:
#from ipywidgets import FloatProgress

In [ ]:
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

In [ ]:
base_path = '../../../dialog_data/dailydialog/'

def read_data(data_path):
    src = []
    trg = []
    with open(base_path+data_path) as f:
        train_data = f.readlines() #f.read().splitlines()
        #print(len(train_data))
        for line in train_data:
            utterences = line.split('__eou__')
            del utterences[-1] #last item is '\n'
            src_utterence = []
            for idx in range(len(utterences)-1):
                # src_utterence=src_utterence+'</s>'+trg_utterence
                src_utterence.append(utterences[idx])
                
                if(len(src_utterence)>5):
                   src_utterence = src_utterence[-5:]
                
                src.append('</s>'.join(src_utterence))
                trg_utterence = utterences[idx+1]
                #trg_utterence = trg_polite_dict[utterences[idx+1]]
                trg.append(trg_utterence)
                
        return src,trg
train_src, train_trg = read_data('train/dialogues_train.txt')
print(len(train_src), len(train_trg))
dev_src, dev_trg = read_data('validation/dialogues_validation.txt')
print(len(dev_src), len(dev_trg))
test_src, test_trg = read_data('test/dialogues_test.txt')
print(len(test_src), len(test_trg))

In [ ]:
# def write_res(lines,name):
#     with open(name, 'w') as f:
#         for line in lines:
#             f.write(f"{line}\n")

# write_res(train_trg,'train_res')
# write_res(dev_trg,'dev_res')
# write_res(test_trg,'test_res')

In [ ]:
# train_src = train_src[0:2]
# train_trg = train_trg[0:2]
# dev_src = dev_src[0:2]
# dev_trg = dev_trg[0:2]

In [ ]:
with open(base_path + 'test/dialogues_test.txt') as myfile:
    head = [next(myfile) for x in range(1)]
print(head)

length = len(head[0].split('__eou__'))

print('\n')
for idx in range(int(length/2)):
    print(test_src[idx])
    print(test_trg[idx])
    print('\n')

In [ ]:
import pandas as pd

train_df = pd.DataFrame(
    {'src': train_src,
     'trg': train_trg
    })
train_df.head()

dev_df = pd.DataFrame(
    {'src': dev_src,
     'trg': dev_trg
    })
dev_df.head()

test_df = pd.DataFrame(
    {'src': test_src,
     'trg': test_trg
    })
test_df.head()

In [ ]:
# model_name = "90MBB_facebook/blenderbot_small-90M/checkpoint-87000/"
# model_name = "facebook/blenderbot_small-90M"
# model_name = 'facebook/blenderbot-400M-distill/checkpoint-15615'
model_name = 'facebook/blenderbot-400M-distill'

In [ ]:
from tqdm.notebook import tqdm
# from transformers import BlenderbotSmallTokenizer
from transformers import BlenderbotTokenizer
import torch

tokenizer = BlenderbotTokenizer.from_pretrained(model_name)
#tokenizer = BlenderbotSmallTokenizer.from_pretrained(model_name)

In [ ]:
from transformers import BlenderbotForConditionalGeneration
# from transformers import BlenderbotSmallForConditionalGeneration
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer
model = BlenderbotForConditionalGeneration.from_pretrained(model_name).to(device)
#model = BlenderbotSmallForConditionalGeneration.from_pretrained(model_name).to(device)

In [ ]:
def gen(src):
    src_tknz = tokenizer(src, truncation=True, padding=True, max_length=128, return_tensors='pt')
    generated_ids = model.generate(src_tknz["input_ids"].cuda(), max_length=128)

    return tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

In [ ]:
for idx in range(50):
    print('src:', test_df['src'].values.tolist()[idx])
    print('trg:', test_df['trg'].values.tolist()[idx])

    print('pred:', gen(test_df['src'].values.tolist()[idx]))
    print('\n')

In [ ]:
import pickle

In [ ]:
dlg_responses_pred_wo = []
for idx in range(len(test_df['src'].values.tolist())):
    dlg_responses_pred_wo.append((gen(test_df['src'].values.tolist()[idx])))

print(len(dlg_responses_pred_wo))

with open('dlg_responses_wo_pred_lists.pkl', 'wb') as f:
    pickle.dump(dlg_responses_pred_wo, f)